# 权重初始化

本文主要参考了以下资料以简单了解神经网络权重初始化的相关基本概念。

- [Weight Initialization in Neural Networks: A Journey From the Basics to Kaiming](https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79)
- [Initializing neural networks](https://www.deeplearning.ai/ai-notes/initialization/#IV)

## 为什么要初始化权重

初始化权重是神经网络训练的第一步，合适的权重初始化能防止深层神经网络的前向过程中输出的梯度爆炸/消失。如果出现梯度爆炸/消失，loss的梯度将太大或太小而无法有益地反向传播，并且即使可以做到，NN也将花费更长的时间收敛。

比如将所有权重初始化为0会导致神经元学习到相同的特征。实际上，任何常量式的初始化方式都会表现地很差。考虑一个仅有两个隐含单元的神经网络，假设我们初始化所有biases为0，权重为常数$\alpha$。如果我们前向输入 ($x_1,x_2$)，每个输出都将是 $relu(\alpha x_1 + \alpha x_2)$。每个隐含单元会对cost有相同的影响，这会导致一样的梯度，那么自然神经元值就会以相同的方式变化，这样神经元就很难学习到不同的规律了。

即使初始化权重是变化的，也要注意当权重值初始化过大时会导致发散，过小时会导致学习过慢。下面看一看梯度爆炸和消失的问题。

考虑一个9层的神经网络。

![](img/9layer.png)

假设所有激活函数是线性的，输出有：

$\hat y = a^{[L]}=W^{[L]}W^{[L-1]}W^{[L-2]}\cdots W^{[3]}W^{[2]}W^{[1]}x$

其中，L=10，$W^{[1]}, W^{[2]}, \cdots W^{[L-1]}$都是2\*2矩阵。假设$W^{[1]}=W^{[2]}= \cdots =W^{[L-1]}=W$，那么有 $\hat y=W^{[L]}W^{L-1}x$，其中$W^{L-1}$表示L-1次方。

那么可以想到，当权值过大时，很容易在幂次后变得很大，即cost相对于参数的梯度非常大，这会使cost在最小值周围震荡；而当太小时，则梯度会很快接近0，使得cost还没到最小就不怎么变化了。

下面是一些例子。

In [1]:
import torch

In [2]:
# 标准正态分布
x = torch.randn(512)

假设有100层的神经网络，没有激活函数。每层一个矩阵a包括层的权重。那么就是100个连续矩阵的乘法来执行前向计算。

In [3]:
for i in range(100): 
    a = torch.randn(512,512)
    x = a @ x # @是pytorch中矩阵相乘的运算
x.mean(), x.std()

(tensor(nan), tensor(nan))

可以看到输出太大以至于计算机识别不了均值和标准差了。可以进一步看看：

In [4]:
x = torch.randn(512)

for i in range(100):
    a = torch.randn(512,512)
    x = a @ x
    if torch.isnan(x.std()): break
i

27

可以看到28层就已经梯度爆炸了。

也就是说将权重初始化范围定到和输入数据相同的分布里并不是一个好方法，容易导致梯度爆炸。

梯度消失的情况如下，将标准差缩小到0.01：

In [5]:
x = torch.randn(512)

for i in range(100): 
    a = torch.randn(512,512) * 0.01
    x = a @ x
x.mean(), x.std()

(tensor(0.), tensor(0.))

那么如何选取合适的初始化值呢

## 如何找到合适的初始化值

在前向计算中，假设仅考虑线性变换，则输出y就是输入x和权重a的乘积$y_i=\sum _{k=0}^{n-1}a_{i,k}x_k$，其中i就是权重a的行序号，k是列序号。

可以证明在给定的层，输入x和从标准正态分布初始化的权重矩阵a的矩阵乘积，平均而言，有一个非常接近于输入连接数的平方根的标准差，在上面的实例中就是 $\sqrt{512}$

In [6]:
import math
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512,512)
    y = a @ x
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/10000, math.sqrt(var/10000)

(0.00028589955177158117, 22.625784281824924)

In [7]:
math.sqrt(512)

22.627416997969522

从定义矩阵乘法的角度看待该属性，这个特点并不意外：为了计算y，我们相加了512个 输入x的一个元素与权重a的一列的element-wise 乘积 。在我们使用标准正态分布初始化x和a的示例中，这512个乘积中的每一个乘积均值为0，标准偏差为1。所以512个乘积加起来有均值0和方差512，所以有标准差$\sqrt{512}$

如果我们首次归一化就选择除以$\sqrt{512}$，那么y的输出就会有$1/\sqrt{512}$的方差。

In [8]:
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(1)
    a = torch.randn(1)
    y = a*x
    mean += y.item()
    var += y.pow(2).item()
mean/10000, math.sqrt(var/10000)

(-0.0007510259594317177, 1.0156330218685454)

In [9]:
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(1)
    a = torch.randn(1)*math.sqrt(1./512)
    y = a*x
    mean += y.item()
    var += y.pow(2).item()
mean/10000, var/10000

(0.0005906981339892183, 0.0019873223970937547)

In [10]:
1/512

0.001953125

这意味着矩阵y的标准偏差（包含通过输入x和权重a之间的矩阵乘法生成的 512 个值中的每一个值）将为 1。通过实验进行确认：

In [11]:
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512,512)*math.sqrt(1./512)
    y = a @ x
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/10000, math.sqrt(var/10000)

(0.001038469743024325, 1.0000861601606938)

现在，让我们重新运行我们的100层网络。与以前一样，我们首先选择层的权重从标准正态分布内[-1,1]随机的，但这次我们通过1 /√n 缩放权重

In [12]:
x = torch.randn(512)

for i in range(100):
    a = torch.randn(512,512) * math.sqrt(1./512)
    x = a @ x
x.mean(), x.std()

(tensor(0.0142), tensor(0.7058))

可以看到在100个假设的层之后，不会有爆炸或消失。

## Xavier初始化

现在考虑实际神经网络的初始化。

为了防止梯度消失和爆炸，通常我们会遵守如下的规律：

1. 激活值的均值为0
2. 激活值的方差每层保持不变

在这样两个假设下，反向传播梯度不会变的太大或太小。

更具体地，考虑一个层l。其前向传播是：

$$a^{[l-1]}=g^{[l-1]}(z^{[l-1]})$$
$$z^{[l]}=W^{[l]}(a^{[l-1]})+b^{[l]}$$
$$a^{[l]}=g^{[l]}(z^{[l]})$$

我们希望有：

$$E[a^{[l-1]}]=E[a^{[l]}]$$
$$Var[a^{[l-1]}]=Var[a^{[l]}]$$

确保零均值并保持每一层输入的方差值，可以保证不会梯度爆炸/消失，我们稍后将对此进行解释。

此方法既适用于正向传播(用于activations，即激活函数的输出值)，也适用于反向传播(用于cost相对于activations的梯度)。对于每一层l，推荐的初始化是Xavier初始化(或其派生方法之一)：

$$W^{[l]}\sim \mathscr N(\mu =0,\sigma ^2=\frac{1}{n^{[l-1]}})$$
$$b^{[l]}=0$$

也就是说，l层的所有权值都随机地从均值为$\mu =0$，方差为$\sigma^2 = \frac{1}{n^{[l-1]}}$ 的正态分布中选取，其中$n^{[l-1]}$是l-1层神经元的数目。偏差用零初始化。

下面是例子，激活函数使用tanh

In [13]:
def tanh(x): 
    return torch.tanh(x)

In [14]:
x = torch.randn(512)

for i in range(100): 
    a = torch.randn(512,512) * math.sqrt(1./512)
    x = tanh(a @ x)
x.mean(), x.std()

(tensor(0.0028), tensor(0.0680))

可以看到100层之后，激活函数输出的标准差约为0.06，这有些小了，不过激活值没有消失。

此方法直到Xavier等人的2010年文章都不是“标准”方法，此前人们还是用的-1到1均匀分布后用$1/\sqrt n$scaling

In [15]:
x = torch.randn(512)

for i in range(100): 
    a = torch.Tensor(512,512).uniform_(-1, 1) * math.sqrt(1./512)
    x = tanh(a @ x)
x.mean(), x.std()

(tensor(-3.2154e-26), tensor(1.0737e-24))

可以看到这种方法基本上activations就消失了。

从数学论证角度简单了解Xavier Initialization 可以参考[这里](https://www.deeplearning.ai/ai-notes/initialization/#IV)

实践中，机器学习工程师也会用$\mathscr N(\mu =0,\sigma ^2=\frac{1}{n^{[l-1]}})$或者$\mathscr N(\mu =0,\sigma ^2=\frac{2}{n^{[l-1]+n^{[l]}}})$来初始化作为Xavier 初始化。

根据[这个blog](https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79)的介绍，Xavier论文里用的是$\pm\frac{\sqrt 6}{\sqrt{n_i+n_{i+1}}}$，其中$n_i$是入层fan-in的神经元个数，$n_{i+1}$是fan-out的神经元个数。

In [16]:
def xavier(m,h): 
    return torch.Tensor(m, h).uniform_(-1, 1)*math.sqrt(6./(m+h))

In [17]:
x = torch.randn(512)

for i in range(100):
    a = xavier(512, 512)
    x = tanh(a @ x)
x.mean(), x.std()

(tensor(-0.0023), tensor(0.0657))

## Kaiming 初始化

对于非对称的激活函数，比如ReLU，更常用的是Kaiming初始化方法。下面简单看看例子。

In [18]:
def relu(x): return x.clamp_min(0.)

In [19]:
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512,512)
    y = relu(a @ x)
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/10000, math.sqrt(var/10000)

(9.022719645738603, 16.00082034854704)

结果是，当使用reu激活时，单层的平均标准差将非常接近输入连接数的平方根，除以2的平方根，在我们的例子中是√512/√2。

In [20]:
math.sqrt(512/2)

16.0

将权重矩阵a的值按这个数字缩放将使每个ReLU层的平均标准差为1。

In [21]:
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512,512) * math.sqrt(2/512)
    y = relu(a @ x)
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/10000, math.sqrt(var/10000)

(0.5644591210246086, 1.000717541461769)

正如之前所展示的，将层激活的标准偏差保持在1左右将允许我们在深度神经网络中堆叠更多的层，而不会发生梯度爆炸或消失。再和Xavier方法对比下：

In [22]:
def kaiming(m,h): 
    return torch.randn(m,h)*math.sqrt(2./m)

In [23]:
x = torch.randn(512)

for i in range(100): 
    a = kaiming(512, 512)
    x = relu(a @ x)
x.mean(), x.std()

(tensor(0.3214), tensor(0.4873))

In [24]:
x = torch.randn(512)

for i in range(100):
    a = xavier(512, 512)
    x = relu(a @ x)
x.mean(), x.std()

(tensor(3.3556e-16), tensor(4.8216e-16))

可以看到在ReLU中，Xavier方法是会出现激活值消失的。